* set a context TF (D1,H4)
* Refresh all CDS from context to higher tf
* get a list of higher columns to merge


In [39]:
from jgtpy import JGTCDSSvc as svc

In [40]:
i="SPX500"
i="EUR/USD"
t="D1"
t="H4"

In [41]:
from jgtutils import jgtpov as jpov

In [42]:
povs=jpov.get_higher_tf_array(t)

In [43]:
use_fresh=True

In [44]:
use_full = False
quotescount=333
if use_full:
  quotescount=-1



In [45]:


for pov in povs:
  print(pov)

H4
D1
W1
M1


In [46]:

cdf_datasets=svc.get_higher_cdf_datasets(i,t,use_full=use_full,use_fresh=use_fresh,quotescount=quotescount,quiet=True)

CDSSvc Get:  H4  of :  H4
CDSSvc Get:  D1  of :  H4
CDSSvc Get:  W1  of :  H4
CDSSvc Get:  M1  of :  H4


In [47]:
for k in cdf_datasets:
  
  #print(cdf)
  print(k)
  v=cdf_datasets[k]
  #print(v.columns)
  #print(v.tail(1))
  #print("")

H4
D1
W1
M1


In [48]:
from jgtutils.jgtconstants import (MFI_VAL,ZCOL,AO)
columns_to_get_from_higher_tf = [MFI_VAL,ZCOL,AO]


In [49]:

workset=cdf_datasets.copy()
ttf=workset[t]
ttf.tail(2)

,BidOpen,BidHigh,BidLow,BidClose,AskOpen,AskHigh,AskLow,AskClose,Volume,Open,...,price_peak_above,price_peak_bellow,ao_peak_above,ao_peak_bellow,mfi_sq,mfi_green,mfi_fade,mfi_fake,mfi_sig,mfi_str
Date,,,,,,,,,,,,,,,,,,,,,
2024-06-18 09:00:00,1.07193,1.07341,1.07099,1.07310,1.07197,1.07346,1.07101,1.07312,41090,1.07195,...,0,0,0,0,1,0,0,0,4,+-
2024-06-18 13:00:00,1.07310,1.07334,1.07300,1.07301,1.07312,1.07336,1.07301,1.07302,674,1.07311,...,0,0,0,0,0,0,0,1,3,-+


In [50]:
created_columns=[]
for c in columns_to_get_from_higher_tf:
  for k in workset:
    if not c in created_columns: 
      created_columns.append(c)
    new_col_name = c+"_"+k
    if k != t:
      if not new_col_name in created_columns: 
        created_columns.append(new_col_name)
print(created_columns)


['mfi_str', 'mfi_str_D1', 'mfi_str_W1', 'mfi_str_M1', 'zcol', 'zcol_D1', 'zcol_W1', 'zcol_M1', 'ao', 'ao_D1', 'ao_W1', 'ao_M1']


In [51]:

for k in workset:  
  if k!=t:
    v=workset[k]
    for c in columns_to_get_from_higher_tf:
     
      new_col_name = c+"_"+k
      ttf[new_col_name]=None

      for ii, row in ttf.iterrows():
        #get the date of the current row (the index)
        date = ii
        #print(k)
        data = v[v.index <= date]
        if not data.empty:
          data = data.iloc[-1]
          ttf.at[ii,new_col_name]=data[c]
        
      



In [52]:
print(created_columns)
#order the columns
#created_columns.sort()
#print(created_columns)


['mfi_str', 'mfi_str_D1', 'mfi_str_W1', 'mfi_str_M1', 'zcol', 'zcol_D1', 'zcol_W1', 'zcol_M1', 'ao', 'ao_D1', 'ao_W1', 'ao_M1']


In [53]:
#Print the ttf
#print(ttf.tail(5))

columns_we_want_to_keep_to_view="Volume,Open,High,Low,Close,ao,ac,jaw,teeth,lips,bjaw,bteeth,blips,tjaw,tteeth,tlips,fh,fl,fdbb,fdbs,fdb,zlc,zcol,mfi_str,M1_mfi_str,W1_mfi_str,M1_zcol,W1_zcol,W1_ao,M1_ao"
columns_we_want_to_keep_to_view="zcol,W1_zcol,M1_zcol,mfi_str,W1_mfi_str,M1_mfi_str,ao,W1_ao,M1_ao"
columns_we_want_to_keep_to_view=created_columns

#select columns in ttf
try:
  ttf_sel=ttf[columns_we_want_to_keep_to_view.split(",")].copy()
except:
  ttf_sel=ttf[columns_we_want_to_keep_to_view].copy()
  

ttf_sel.to_csv("ttf_sel.csv")

In [54]:
# ttf=daily_df
# for ii, row in daily_df.iterrows():
#   #get the date of the current row (the index)
#   date = ii
#   #print(date)
#   for k in workset:  
#     if k!=t:
#       for c in columns_to_get_from_higher_tf:
#         new_col_name = k+"_"+c
#         ttf[new_col_name]=None
#         #print(k)
#         v=workset[k]
#         data = v[v.index <= date]
#         if not data.empty:
#           data = data.iloc[-1]
#           ttf.at[ii,new_col_name]=data[c]
        
      

In [55]:

ttf.columns

Index(['BidOpen', 'BidHigh', 'BidLow', 'BidClose', 'AskOpen', 'AskHigh',
       'AskLow', 'AskClose', 'Volume', 'Open', 'High', 'Low', 'Close',
       'Median', 'ao', 'ac', 'jaw', 'teeth', 'lips', 'bjaw', 'bteeth', 'blips',
       'tjaw', 'tteeth', 'tlips', 'fh', 'fl', 'fh3', 'fl3', 'fh5', 'fl5',
       'fh8', 'fl8', 'fh13', 'fl13', 'fh21', 'fl21', 'fh34', 'fl34', 'fh55',
       'fl55', 'fh89', 'fl89', 'mfi', 'fdbb', 'fdbs', 'fdb', 'aoaz', 'aobz',
       'zlc', 'zlcb', 'zlcs', 'zcol', 'sz', 'bz', 'acs', 'acb', 'ss', 'sb',
       'price_peak_above', 'price_peak_bellow', 'ao_peak_above',
       'ao_peak_bellow', 'mfi_sq', 'mfi_green', 'mfi_fade', 'mfi_fake',
       'mfi_sig', 'mfi_str', 'mfi_str_D1', 'zcol_D1', 'ao_D1', 'mfi_str_W1',
       'zcol_W1', 'ao_W1', 'mfi_str_M1', 'zcol_M1', 'ao_M1'],
      dtype='object')

In [56]:
ttf.tail(1)

,BidOpen,BidHigh,BidLow,BidClose,AskOpen,AskHigh,AskLow,AskClose,Volume,Open,...,mfi_str,mfi_str_D1,zcol_D1,ao_D1,mfi_str_W1,zcol_W1,ao_W1,mfi_str_M1,zcol_M1,ao_M1
Date,,,,,,,,,,,,,,,,,,,,,
2024-06-18 13:00:00,1.0731,1.07334,1.073,1.07301,1.07312,1.07336,1.07301,1.07302,674,1.07311,...,-+,-+,gray,-0.338136,-+,red,-0.035929,-+,gray,0.000463


* Iterations takes 1 min

## Above we are getting the columns from higher TF

In [57]:
    

# for k in workset:  
#   if k!=t:
#     print(k)
#     v=workset[k]
#     for c in columns_to_get_from_higher_tf:
#       new_col_name = k+"_"+c
#       print("Adding column for "+c+ " from tf:"+k + " as :"+new_col_name)
#       daily_df[new_col_name]=None
#       for ii, row in daily_df.iterrows():
#         date = ii
#         #print(date)
#         # Find the latest monthly data before or on the current date
#         v_data = v[v['Date'] <= date].iloc[-1]
#         daily_df.at[i, new_col_name] = v_data[c]
        
      
  
  